<a href="https://colab.research.google.com/github/mrhamedani/LLM-Agents/blob/main/10_bleu_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Evaluating translations with BLEU
Models: nllb-200-distilled-600M

Colab environment: CPU.

Keys:
* Bleu Evaluation.
* Translation Pipeline.
* Google Translator API.




In [ ]:
import os
os.kill(os.getpid(), 9)

In this notebook, we will use the BLEU metric to compare the quality of two different approaches for performing translations.


In [1]:
# Install the correct versions of packages
!pip install -q numpy==1.26.4
!pip install -q transformers==4.42.1 evaluate==0.4.2 googletrans==3.1.0a0


In [2]:
from googletrans import Translator
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import sacrebleu

In [11]:
#Sentences to Translate.
sentences = [
    "در فصل‌های قبلی، بیشتر با مدل‌های OpenAI کار کردید و با مدل‌های متن‌باز Hugging Face، استفاده از تعبیه‌ها، پایگاه‌های داده برداری و ایجنت‌ها آشنا شدید.",
    "این فصل‌ها بسیار کاربردی بودند و سعی کردم مفاهیمی را به‌تدریج معرفی کنم که به شما کمک کرده‌اند دانش خود را گسترش دهید و پروژه‌هایی با استفاده از فناوری‌های مدرن مدل‌های زبانی بزرگ ایجاد کنید."
]


In [12]:

# Reference translations (for BLEU score calculation)
reference_translations = [
    ["In the previous chapters, you mostly worked with OpenAI models and got a practical introduction to Hugging Face's open-source models, the use of embeddings, vector databases, and agents."],
    ["These were very practical chapters where I tried to gradually introduce concepts that helped you expand your knowledge and start building projects using the modern technology stack of large language models."]
]


##Create Translations with NLLB.
We will perform the first translation using the NLLB model, a small model specialized in performing translations, which we will retrieve from Hugging Face.

When creating the pipeline, we pass the source language and the target language of the translation to it.

In [13]:
model_id = "facebook/nllb-200-distilled-600M"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [14]:
translator_nllb = pipeline('translation', model=model, tokenizer=tokenizer,
                           src_lang="fas_Arab", tgt_lang="eng_Latn")

# ترجمه با مدل NLLB
translations_nllb = []
for text in sentences:
    print("To translate (NLLB):", text)
    translation = translator_nllb(text)
    translations_nllb.append(translation[0]['translation_text'])
    print(translation[0]['translation_text'])

To translate (NLLB): در فصل‌های قبلی، بیشتر با مدل‌های OpenAI کار کردید و با مدل‌های متن‌باز Hugging Face، استفاده از تعبیه‌ها، پایگاه‌های داده برداری و ایجنت‌ها آشنا شدید.
In previous seasons, you've mostly worked with OpenAI models and have been very familiar with open-source Hugging Face models, the use of packages, databases and agents.
To translate (NLLB): این فصل‌ها بسیار کاربردی بودند و سعی کردم مفاهیمی را به‌تدریج معرفی کنم که به شما کمک کرده‌اند دانش خود را گسترش دهید و پروژه‌هایی با استفاده از فناوری‌های مدرن مدل‌های زبانی بزرگ ایجاد کنید.
These chapters were very useful and I tried to introduce concepts gradually that have helped you expand your knowledge and create projects using modern technologies, large language models.


##Create Translations with Google Traslator.

As a second source for translations, we will use the Google Translator API.

In [15]:

translator_google = Translator()
translations_google = []
for text in sentences:
    print("To translate (Google):", text)
    translation = translator_google.translate(text, dest="en")
    translations_google.append(translation.text)
    print(translation.text)

To translate (Google): در فصل‌های قبلی، بیشتر با مدل‌های OpenAI کار کردید و با مدل‌های متن‌باز Hugging Face، استفاده از تعبیه‌ها، پایگاه‌های داده برداری و ایجنت‌ها آشنا شدید.
In the previous chapters, you were mostly working with Openai models and familiar with the Hugging Face opening models, the use of embeddeds, databases and agents.
To translate (Google): این فصل‌ها بسیار کاربردی بودند و سعی کردم مفاهیمی را به‌تدریج معرفی کنم که به شما کمک کرده‌اند دانش خود را گسترش دهید و پروژه‌هایی با استفاده از فناوری‌های مدرن مدل‌های زبانی بزرگ ایجاد کنید.
These chapters were very practical and I tried to gradually introduce concepts that have helped you expand your knowledge and create projects using modern technologies of large language models.


In this list, we have the translations created by Google.

## Evaluate translations with BLEU

We will use the BLEU implementation from the Evaluate library by Hugging Face.

In [17]:
# Calculate BLEU scores using sacrebleu
# Calculate BLEU for NLLB model
bleu_nllb = sacrebleu.corpus_bleu (translations_nllb, [reference_translations])
print("BLEU Score for NLLB Model: ", bleu_nllb.score)

# Calculate BLEU for Google Translate
bleu_google = sacrebleu.corpus_bleu(translations_google, [reference_translations])
print("BLEU Score for Google Translate: ", bleu_google.score)


TypeError: BLEU: `refs` should be a sequence of sequence of strings.

It appears that the translation performed by the Google API is significantly better than the one performed by the NLLB model.